In [38]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import datetime as dt
import math

/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Se obtienen los datasets de entrenamiento y test y se los transforma en dataframes para su análisis.

In [2]:
df_train = pd.read_csv('bike_train.csv')
df_test = pd.read_csv('bike_test.csv')

In [3]:
df_test.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,12000,5/20/2012,2,1,5,4,0,0,0,1,0.52,0.5000,0.68,0.0896
1,12001,5/20/2012,2,1,5,5,0,0,0,1,0.50,0.4848,0.72,0.1045
2,12002,5/20/2012,2,1,5,6,0,0,0,1,0.50,0.4848,0.63,0.1343
3,12003,5/20/2012,2,1,5,7,0,0,0,1,0.52,0.5000,0.68,0.1940
4,12004,5/20/2012,2,1,5,8,0,0,0,1,0.56,0.5303,0.56,0.1642


In [116]:
df_train.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1/1/2011,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,1/1/2011,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,1/1/2011,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,1/1/2011,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,1/1/2011,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


El dataset de test no posee la columna casual, registered ni cnt

Se transforma la columna fecha en un formato ordinal para ser utilizada posteriormente en los modelos elegidos

In [4]:
train = df_train.copy()
test = df_test.copy()
train.dteday = pd.to_datetime(train.dteday)
train.dteday = train.dteday.map(dt.datetime.toordinal)
test.dteday = pd.to_datetime(test.dteday)
test.dteday = test.dteday.map(dt.datetime.toordinal)

Se plantea la matriz de correlación correspondiente para observar la relación entre las distintas features

In [11]:
train.corr().style.background_gradient(cmap='Greens')

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,1.000000,0.999990,0.118039,0.777284,0.167195,-0.007446,0.008323,0.007103,0.005295,-0.015359,0.029297,0.042490,-0.013275,-0.029001,0.115768,0.238546,0.229814
dteday,0.999990,1.000000,0.119277,0.776468,0.168474,-0.009457,0.008254,0.007121,0.005410,-0.015217,0.030437,0.043644,-0.012197,-0.029317,0.115635,0.238038,0.229357
season,0.118039,0.119277,1.000000,-0.446665,0.865268,-0.007610,-0.008928,-0.003688,0.012630,-0.009029,0.410790,0.413196,0.198937,-0.142924,0.119043,0.093595,0.111369
yr,0.777284,0.776468,-0.446665,1.000000,-0.488103,-0.003948,0.001039,-0.002714,0.001068,-0.006309,-0.198929,-0.190360,-0.141319,0.067356,0.053371,0.171910,0.156960
mnth,0.167195,0.168474,0.865268,-0.488103,1.000000,-0.007060,0.009372,0.014500,0.006441,-0.012239,0.353330,0.358541,0.201669,-0.147042,0.078563,0.061686,0.073431
hr,-0.007446,-0.009457,-0.007610,-0.003948,-0.007060,1.000000,0.000414,-0.004037,0.003454,-0.024003,0.139251,0.134633,-0.257202,0.130441,0.285182,0.381933,0.396698
holiday,0.008323,0.008254,-0.008928,0.001039,0.009372,0.000414,1.000000,-0.102503,-0.248601,-0.014707,-0.028519,-0.038005,-0.036672,0.022681,0.022140,-0.044534,-0.030304
weekday,0.007103,0.007121,-0.003688,-0.002714,0.014500,-0.004037,-0.102503,1.000000,0.031057,0.017259,-0.010002,-0.009841,-0.025932,0.008447,0.023687,0.019179,0.022618
workingday,0.005295,0.005410,0.012630,0.001068,0.006441,0.003454,-0.248601,0.031057,1.000000,0.042202,0.062273,0.066305,0.022805,-0.002493,-0.298241,0.134789,0.025132
weathersit,-0.015359,-0.015217,-0.009029,-0.006309,-0.012239,-0.024003,-0.014707,0.017259,0.042202,1.000000,-0.080149,-0.081386,0.430378,0.009195,-0.147903,-0.130223,-0.149848


A partir de lo observado se decide utilizar las features que aparecen a continuación y definir como variable objetivo la cantidad de bicicletas rentadas

In [5]:
x = train.drop(columns=['cnt', 'registered', 'casual', 'temp', 'mnth', 'yr', 'dteday'])
y = train.cnt
x_predecir = test.drop(columns=['temp', 'mnth', 'yr', 'dteday'])

Se dividen estos datos para poder realizar pruebas en cada uno de los modelos

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

### Se presentan análisis sobre disitintas soluciones o modelos para definir cuál es el que presenta un menor RMSE

#### Decision Tree Regresor

In [93]:
tree_para = {'criterion':['squared_error']}
model_dtr = GridSearchCV(DecisionTreeRegressor(), tree_para, cv=5)

In [94]:
model_dtr.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error']})

In [95]:
model_dtr.cv_results_

{'mean_fit_time': array([0.03177729]),
 'std_fit_time': array([0.00891469]),
 'mean_score_time': array([0.00153542]),
 'std_score_time': array([2.20343144e-05]),
 'param_criterion': masked_array(data=['squared_error'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'squared_error'}],
 'split0_test_score': array([0.86275722]),
 'split1_test_score': array([0.88690157]),
 'split2_test_score': array([0.88518543]),
 'split3_test_score': array([0.88076273]),
 'split4_test_score': array([0.88393372]),
 'mean_test_score': array([0.87990813]),
 'std_test_score': array([0.00880735]),
 'rank_test_score': array([1], dtype=int32)}

In [96]:
y_pred = model_dtr.predict(x_test)
RMSE_tree = math.sqrt(mean_squared_error(y_test, y_pred))
RMSE_tree

49.651590273290005

#### Random Forest Regresor

In [97]:
randomf_para = {'criterion':['squared_error']}
model_rfr = GridSearchCV(RandomForestRegressor(), randomf_para, cv=5)

In [98]:
model_rfr.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'criterion': ['squared_error']})

In [99]:
model_rfr.cv_results_

{'mean_fit_time': array([1.77241731]),
 'std_fit_time': array([0.03842259]),
 'mean_score_time': array([0.03842845]),
 'std_score_time': array([0.00148721]),
 'param_criterion': masked_array(data=['squared_error'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'squared_error'}],
 'split0_test_score': array([0.92660757]),
 'split1_test_score': array([0.93655458]),
 'split2_test_score': array([0.93782362]),
 'split3_test_score': array([0.93649844]),
 'split4_test_score': array([0.93988278]),
 'mean_test_score': array([0.9354734]),
 'std_test_score': array([0.00459975]),
 'rank_test_score': array([1], dtype=int32)}

In [100]:
y_pred = model_rfr.predict(x_test)
RMSE_tree = math.sqrt(mean_squared_error(y_test, y_pred))
RMSE_tree

35.85103784295047

#### XGBRegressor

In [101]:
xgbr_para = {}
model_xgbr = GridSearchCV(XGBRegressor(), xgbr_para, cv=5)

In [102]:
model_xgbr.fit(x_train, y_train)

/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Ind

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             param_grid={})

In [103]:
model_xgbr.cv_results_

{'mean_fit_time': array([0.35107503]),
 'std_fit_time': array([0.22567911]),
 'mean_score_time': array([0.00258117]),
 'std_score_time': array([0.00046621]),
 'params': [{}],
 'split0_test_score': array([0.93353922]),
 'split1_test_score': array([0.9487069]),
 'split2_test_score': array([0.94115362]),
 'split3_test_score': array([0.9497131]),
 'split4_test_score': array([0.94814145]),
 'mean_test_score': array([0.94425086]),
 'std_test_score': array([0.00615079]),
 'rank_test_score': array([1], dtype=int32)}

In [104]:
y_pred = model_xgbr.predict(x_test)
RMSE_tree = math.sqrt(mean_squared_error(y_test, y_pred))
RMSE_tree

31.52089126848505

#### ALTERNATIVA DOBLE PREDICCION

Esta alternativa consiste en la elaboración de dos modelos para la predicción que se solicita. El primer modelo se encargara de predecir la feature casual que como se observa en la matriz de correlación presenta una relación bastante evidente con la variable objetivo cnt, pero el dataset de test no cuenta con ésta. Por este motivo en esta primera instancia se realiza esta acción. El segundo modelo se encarga de predecir la variable objetivo a partir del dataset de test mas la columna casual que se predijo con el primer modelo. Esta alternativa solo se realizó con el objetivo de ver el comportamiento de lo que se plantea y no asegura un resultado acorde

Dividir el dataset para realizar el entramiento correspondiente y realizar pruebas

In [72]:
x1 = train.drop(columns=['cnt', 'registered', 'casual', 'temp', 'mnth', 'yr', 'dteday'])
x2 = train.drop(columns=['cnt', 'registered', 'temp', 'mnth', 'yr', 'dteday'])
y2 = train.cnt
y1 = train.casual

In [73]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.2)

Se utilizará para ambos modelos XGBRegressor() porque es el que presento mejor desempeño anteriormente

Entrenamiento de primer modelo

In [74]:
model1 = XGBRegressor()
model1.fit(x_train1, y_train1)
pred1 = model1.predict(x_test1)
e = mean_squared_error(y_test1, pred1)
math.sqrt(e)

/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


11.174232414281114

Entrenamiento de segundo modelo

In [75]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.2)

In [76]:
model2 = XGBRegressor()
model2.fit(x_train2, y_train2)
pred2 = model2.predict(x_test2)
e = mean_squared_error(y_test2, pred2)
math.sqrt(e)

/home/jeo/anaconda3/envs/data/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


26.44031681288778

#### Probar Modelo

Con el modelo entrenado se procede a realizar pruebas para comprobar su rendimiento

In [77]:
x_test1['casual'] = pred1

In [78]:
pred_final = model2.predict(x_test1.sort_values(by=['instant']))

211.35352913804286

In [80]:
valores_finales = train[train.instant.isin(x_test1.instant)]
valores_finales

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
6,7,734138,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
9,10,734138,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14
16,17,734138,1,0,1,16,0,6,0,2,0.42,0.4242,0.82,0.2985,41,52,93
17,18,734138,1,0,1,17,0,6,0,2,0.44,0.4394,0.82,0.2836,15,52,67
23,24,734138,1,0,1,23,0,6,0,2,0.46,0.4545,0.88,0.2985,15,24,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,11975,734642,2,1,5,3,0,6,0,1,0.50,0.4848,0.48,0.1045,10,23,33
11978,11979,734642,2,1,5,7,0,6,0,1,0.46,0.4545,0.67,0.1045,28,67,95
11987,11988,734642,2,1,5,16,0,6,0,1,0.74,0.6515,0.30,0.1045,331,311,642
11990,11991,734642,2,1,5,19,0,6,0,1,0.70,0.6364,0.39,0.1343,203,229,432


In [79]:
pred_final

array([4.5993999e-01, 7.6884331e+01, 8.2136894e+01, ..., 6.3637152e+02,
       3.8334204e+02, 2.2009944e+02], dtype=float32)

In [81]:
e = mean_squared_error(valores_finales.cnt, pred_final)
math.sqrt(e)

30.034373831795396

Se observa una leve disminución del modelo de XGBRegressor por si solo, pero no es significativo dicho cambio

In [86]:
prueba_pred_final = x_predecir.copy()

In [87]:
pred_final1 = model1.predict(prueba_pred_final)

In [88]:
prueba_pred_final['casual'] = pred_final1

In [89]:
pred_final2 = model2.predict(prueba_pred_final)
pred_final2

array([ 64.084694,  97.352745,  92.18333 , ..., 168.04433 , 120.04432 ,
        79.561646], dtype=float32)

Se decide utilizar XGBRegressor para predecir el resultado debido a su menor medida de RMSE

In [105]:
y_final = model_xgbr.predict(x_predecir)

Se aplica valor absoluto a la predicción final para asegurar que no existan valores negativos

In [107]:
final = np.abs(y_final)

In [123]:
resultado = pd.DataFrame(final)
resultado = resultado.rename(columns={0: 'pred'})
resultado

,pred
0,35.846741
1,35.485882
2,61.685482
3,88.390938
4,302.070374
...,...
5375,359.900391
5376,238.535202
5377,195.664307
5378,159.896103


Exportar Resultado a csv

In [124]:
resultado.to_csv('jeo10.csv', index=False)